In [102]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import datetime


In [103]:
# Setup basedir
BASE_DIR = '/Volumes/SANDISK_SSD_G40/GoogleDrive/GitHub/sagehen_meadows'
os.chdir(BASE_DIR)
os.getcwd()

'/Volumes/SANDISK_SSD_G40/GoogleDrive/GitHub/sagehen_meadows'

In [104]:
# Import WRCC/DRI Weather Data
wrcc = pd.read_csv(
    'data/station_instrumentation/climate/WRCC_DRI/RAW/SagehenCk_WeatherData_WRCC_DRI.txt',
    sep='\s+',
    skiprows=4,
    header=None)
wrcc.columns = [
    "datetime",
    "precip_in",
    "wind_speed_mph",
    "wind_dir_deg",
    "avg_air_temp_F",
    "fuel_temp_F",
    "RH_percent",
    "battery_volts",
    "max_gust_dir_deg",
    "max_gust_speed_mph",
    "solar_rad_wpm2"
]
wrcc.replace(-9999.0, pd.NA)
wrcc["datetime"] = pd.to_datetime(
    wrcc["datetime"],
    format="%y%m%d%H%M"
)
#wrcc["datetime"] = pd.DatetimeIndex(wrcc.datetime)
wrcc = wrcc.set_index("datetime")
wrcc.head()

,precip_in,wind_speed_mph,wind_dir_deg,avg_air_temp_F,fuel_temp_F,RH_percent,battery_volts,max_gust_dir_deg,max_gust_speed_mph,solar_rad_wpm2
datetime,,,,,,,,,,
2020-01-01 00:00:00,0.0,0.7,274.0,24.7,-9999.0,87.2,13.2,-9999.0,1.8,0.0
2020-01-01 00:10:00,0.0,0.6,229.0,24.9,-9999.0,87.0,13.2,-9999.0,2.0,0.0
2020-01-01 00:20:00,0.0,1.5,232.6,25.2,-9999.0,86.4,13.2,-9999.0,3.0,0.0
2020-01-01 00:30:00,0.0,0.5,211.2,25.6,-9999.0,84.5,13.2,-9999.0,1.8,0.0
2020-01-01 00:40:00,0.0,0.3,265.7,25.7,-9999.0,85.4,13.2,-9999.0,1.7,0.0


In [105]:
wrcc.index.min(), wrcc.index.max()

(Timestamp('2020-01-01 00:00:00'), Timestamp('2026-01-21 12:20:00'))

In [106]:
wrcc.columns

Index(['precip_in', 'wind_speed_mph', 'wind_dir_deg', 'avg_air_temp_F',
       'fuel_temp_F', 'RH_percent', 'battery_volts', 'max_gust_dir_deg',
       'max_gust_speed_mph', 'solar_rad_wpm2'],
      dtype='object')

In [107]:
# Drop columns with all NA values (-9999.0)
wrcc = wrcc.loc[:,~(wrcc == -9999.0).all()]
wrcc.columns

Index(['precip_in', 'wind_speed_mph', 'wind_dir_deg', 'avg_air_temp_F',
       'RH_percent', 'battery_volts', 'max_gust_speed_mph', 'solar_rad_wpm2'],
      dtype='object')

In [218]:
## COMPLETENESS VALIDATION
# Set start and end time for reporting on completeness
start = "2017-10-01"
end = "2025-11-10"
wrcc_time = wrcc.loc[start:end]

step = pd.Timedelta("10min")
expected_len = int((wrcc_time.index.max() - wrcc_time.index.min()) / step) + 1
expected_len_inferred = len(wrcc_time)
expected_len - expected_len_inferred # diff reflects some irregularity in the time series

completeness_wrcc = (
    wrcc_time.notna()
        .sum()
        .div(expected_len)
        .mul(100)
        .round(1)
        .rename("percent_complete")
)

completeness_wrcc

precip_in             100.0
wind_speed_mph        100.0
wind_dir_deg          100.0
avg_air_temp_F        100.0
RH_percent            100.0
battery_volts         100.0
max_gust_speed_mph    100.0
solar_rad_wpm2        100.0
Name: percent_complete, dtype: float64

In [109]:
# Import DENDRA Weather Data
dendra = pd.read_csv('data/station_instrumentation/climate/Dendra/RAW/Dendra_Sagehen_2010-2025.csv')
dendra["time"] = pd.DatetimeIndex(dendra.time)
dendra = dendra.set_index("time")
dendra.index.min(), dendra.index.max()


(Timestamp('2010-01-01 00:00:00'), Timestamp('2026-01-06 14:40:00'))

In [110]:
dendra.columns

Index(['air-temp-min-degc', 'air-temp-max-degc', 'air-temp-avg-degc',
       'barometric-pressure-avg-mb', 'battery-voltage-avg-v',
       'battery-voltage-max-v', 'battery-voltage-min-v',
       'precipitation-geonor-freq-s-1', 'precipitation-geonor-cm-cm',
       'precipitation-geonor-freq-std-s-1', 'precipitation-geonor-mm-mm',
       'rainfall-mm', 'relative-humidity-max-pct', 'relative-humidity-avg-pct',
       'relative-humidity-min-pct', 'snow-depth-max-in', 'snow-depth-min-in',
       'snow-depth-std-in', 'snow-depth-in', 'snow-temp-avg-degc',
       'snow-temp-min-degc', 'snow-temp-max-degc', 'soil-temp-max-degc',
       'soil-temp-avg-degc', 'soil-temp-min-degc', 'tdr-freq-1-usec',
       'total-solar-radiation-avg-wm-2', 'wind-direction-100-ft-std-deg',
       'wind-direction-100-ft-avg-deg', 'wind-direction-25-ft-std-deg',
       'wind-direction-25-ft-avg-deg', 'wind-speed-100-ft-max-ms',
       'wind-speed-25-ft-max-ms', 'wind-speed-100-ft-avg-ms',
       'wind-speed-25-ft

In [183]:
## COMPLETENESS VALIDATION
# Set start and end time for reporting on completeness
start = "2017-10-01"
end = "2025-11-10"
dendra_time = dendra.loc[start:end]



In [184]:
expected_len_inferred = len(dendra_time)
expected_len_inferred

426261

In [185]:
dendra_time.index.is_monotonic_increasing


True

In [186]:
dendra_time.index.to_series().diff().value_counts().head()


time
0 days 00:10:00    426243
0 days 00:50:00         3
0 days 00:20:00         3
0 days 01:10:00         2
0 days 00:40:00         2
Name: count, dtype: int64

In [187]:
freq = pd.infer_freq(dendra_time.index)
freq
# if nothing returns, pandas cannot infer the frequency of the data

In [188]:
step = pd.to_timedelta(freq)
step
# if nothing returns, pandas cannot infer the timestep of the data

In [189]:
step = pd.Timedelta("10min")
expected_len = int((dendra_time.index.max() - dendra_time.index.min()) / step) + 1
expected_len - expected_len_inferred # diff reflects some irregularity in the time series

411

In [190]:
expected_len

426672

In [191]:
completeness_dendra = (
    dendra_time.notna()
        .sum()
        .div(expected_len)
        .mul(100)
        .round(1)
        .rename("percent_complete")
)

completeness_dendra

air-temp-min-degc                        99.9
air-temp-max-degc                        99.9
air-temp-avg-degc                        99.9
barometric-pressure-avg-mb               99.9
battery-voltage-avg-v                    99.9
battery-voltage-max-v                    99.9
battery-voltage-min-v                    99.9
precipitation-geonor-freq-s-1            99.9
precipitation-geonor-cm-cm                6.9
precipitation-geonor-freq-std-s-1        34.3
precipitation-geonor-mm-mm               34.3
rainfall-mm                               6.9
relative-humidity-max-pct                99.9
relative-humidity-avg-pct                99.9
relative-humidity-min-pct                99.9
snow-depth-max-in                         0.0
snow-depth-min-in                         0.0
snow-depth-std-in                         0.0
snow-depth-in                            97.4
snow-temp-avg-degc                        0.0
snow-temp-min-degc                        0.0
snow-temp-max-degc                

In [192]:
# Report on GOOD-to-GO variables
complete = completeness_dendra[completeness_dendra > 92]
complete

air-temp-min-degc                        99.9
air-temp-max-degc                        99.9
air-temp-avg-degc                        99.9
barometric-pressure-avg-mb               99.9
battery-voltage-avg-v                    99.9
battery-voltage-max-v                    99.9
battery-voltage-min-v                    99.9
precipitation-geonor-freq-s-1            99.9
relative-humidity-max-pct                99.9
relative-humidity-avg-pct                99.9
relative-humidity-min-pct                99.9
snow-depth-in                            97.4
total-solar-radiation-avg-wm-2           99.9
wind-direction-100-ft-std-deg            99.9
wind-direction-100-ft-avg-deg            99.9
wind-direction-25-ft-std-deg             99.9
wind-direction-25-ft-avg-deg             99.9
wind-speed-100-ft-max-ms                 99.9
wind-speed-25-ft-max-ms                  99.9
wind-speed-100-ft-avg-ms                 99.9
wind-speed-25-ft-avg-ms                  99.9
wind-speed-vector-magnitude-25-ft-

In [193]:
incomplete_dendra = completeness_dendra[completeness_dendra <= 92]
incomplete_dendra

precipitation-geonor-cm-cm                6.9
precipitation-geonor-freq-std-s-1        34.3
precipitation-geonor-mm-mm               34.3
rainfall-mm                               6.9
snow-depth-max-in                         0.0
snow-depth-min-in                         0.0
snow-depth-std-in                         0.0
snow-temp-avg-degc                        0.0
snow-temp-min-degc                        0.0
snow-temp-max-degc                        0.0
soil-temp-max-degc                        0.0
soil-temp-avg-degc                        0.0
soil-temp-min-degc                        0.0
tdr-freq-1-usec                          34.3
rainfall-cumulative-mm                    6.9
leaf-wetness-max-mv                      13.9
leaf-wetness-avg-mv                      13.9
leaf-wetness-min-mv                      13.9
leaf-wetness-minutes-contaminated-min    13.9
leaf-wetness-minutes-dry-min             13.9
leaf-wetness-minutes-wet-min             13.9
leaf-wetness-mv                   

In [194]:
dendra_time.columns

Index(['air-temp-min-degc', 'air-temp-max-degc', 'air-temp-avg-degc',
       'barometric-pressure-avg-mb', 'battery-voltage-avg-v',
       'battery-voltage-max-v', 'battery-voltage-min-v',
       'precipitation-geonor-freq-s-1', 'precipitation-geonor-cm-cm',
       'precipitation-geonor-freq-std-s-1', 'precipitation-geonor-mm-mm',
       'rainfall-mm', 'relative-humidity-max-pct', 'relative-humidity-avg-pct',
       'relative-humidity-min-pct', 'snow-depth-max-in', 'snow-depth-min-in',
       'snow-depth-std-in', 'snow-depth-in', 'snow-temp-avg-degc',
       'snow-temp-min-degc', 'snow-temp-max-degc', 'soil-temp-max-degc',
       'soil-temp-avg-degc', 'soil-temp-min-degc', 'tdr-freq-1-usec',
       'total-solar-radiation-avg-wm-2', 'wind-direction-100-ft-std-deg',
       'wind-direction-100-ft-avg-deg', 'wind-direction-25-ft-std-deg',
       'wind-direction-25-ft-avg-deg', 'wind-speed-100-ft-max-ms',
       'wind-speed-25-ft-max-ms', 'wind-speed-100-ft-avg-ms',
       'wind-speed-25-ft

In [195]:
def missing_runs(series):
    is_missing = series.isna()

    # label consecutive True runs
    groups = is_missing.ne(is_missing.shift()).cumsum()

    runs = (
        series[is_missing]
        .groupby(groups)
        .apply(lambda x: pd.DataFrame({
            "start": [x.index[0]],
            "end": [x.index[-1]],
            "n_points": [len(x)]
        }))
        .reset_index(drop=True)
    )

    return runs


In [196]:
missing_runs(dendra_time["precipitation-geonor-mm-mm"])


,start,end,n_points
0,2017-12-18 11:10:00,2017-12-18 11:10:00,1
1,2020-07-14 03:10:00,2025-11-10 23:50:00,279794


In [197]:
precip_runs = missing_runs(dendra_time["precipitation-geonor-cm-cm"])
precip_runs.dtypes

start       datetime64[ns]
end         datetime64[ns]
n_points             int64
dtype: object

In [198]:
runs = precip_runs
runs["duration"] = (runs["n_points"] - 1) * step
MIN_GAP = pd.Timedelta("1D")
major = runs[runs["duration"] >= MIN_GAP]
major

,start,end,n_points,duration
1,2018-04-24 16:00:00,2025-11-10 23:50:00,396645,2754 days 11:20:00


In [199]:
## Identify dates of major gaps for variables with incomplete data
#  Major gap defined as ? days
MIN_GAP = pd.Timedelta("3D")

# Create a dict that reports on gaps
gap_report = []

# Summarize the gaps for columns with major incompleteness issues
incomplete = completeness[completeness < 95]

for col in incomplete.index:
    
    runs = missing_runs(dendra_time[col])
    if runs.empty: 
        continue
    
    #runs["duration"] = (runs["n_points"] - 1) * step
    runs["duration"] = runs["end"] - runs["start"]
    
    major = runs[runs["duration"] >= MIN_GAP]

    # add rows to gaps_report
    for _, row in major.iterrows():
        gap_report.append({
            "variable": col,
            "percent_complete": incomplete[col],
            "gap_start": row["start"],
            "gap_duration": row["duration"]
        })

summary = pd.DataFrame(gap_report)

summary = summary.sort_values(["variable", "gap_start"]).reset_index(drop=True)
summary

,variable,percent_complete,gap_start,gap_duration
0,air-temp-100-ft-avg-degc,92.9,2017-10-01 00:00:00,205 days 15:50:00
1,air-temp-25-ft-avg-degc,92.9,2017-10-01 00:00:00,205 days 15:50:00
2,leaf-wetness-avg-mv,13.9,2017-10-01 00:00:00,2552 days 11:50:00
3,leaf-wetness-max-mv,13.9,2017-10-01 00:00:00,2552 days 11:50:00
4,leaf-wetness-min-mv,13.9,2017-10-01 00:00:00,2552 days 11:50:00
5,leaf-wetness-minutes-contaminated-min,13.9,2017-10-01 00:00:00,2552 days 11:50:00
6,leaf-wetness-minutes-dry-min,13.9,2017-10-01 00:00:00,2552 days 11:50:00
7,leaf-wetness-minutes-wet-min,13.9,2017-10-01 00:00:00,2552 days 11:50:00
8,leaf-wetness-mv,13.9,2017-10-01 00:00:00,2552 days 11:50:00
9,precipitation-geonor-cm-cm,6.9,2018-04-24 16:00:00,2757 days 07:50:00


In [200]:
# Calculate min, max and mean for a column (or weather variable)
tdr_stats = dendra.loc['2017-10-01':'2020-07-01', 'tdr-freq-1-usec'].agg(["min", "max", "mean"])
tdr_stats

min    -21.40000
max     31.11000
mean     5.13563
Name: tdr-freq-1-usec, dtype: float64

In [201]:
complete_vars = completeness[completeness > 95].index
stats = (dendra[complete_vars].agg(["min", "max", "mean", "std"]).T)
summary_stats = stats.join(completeness)
summary_stats


,min,max,mean,std,percent_complete
air-temp-min-degc,-100.000000,66.33963,5.108864,10.059436,99.9
air-temp-max-degc,-47.450000,33.73000,5.537239,10.169941,99.9
air-temp-avg-degc,-49.800000,33.44000,5.322992,10.110485,99.9
barometric-pressure-avg-mb,0.000000,869.22190,833.679587,32.242398,99.9
battery-voltage-avg-v,-20.076040,15.13000,13.170705,0.286775,99.9
battery-voltage-max-v,8.930000,20.01986,13.205573,0.294279,99.9
battery-voltage-min-v,-1.726680,15.13000,13.157758,0.286840,99.9
precipitation-geonor-freq-s-1,0.000000,5165.00000,1950.746894,572.013537,99.9
relative-humidity-max-pct,0.000000,100.00000,66.329945,24.972351,99.9
relative-humidity-avg-pct,0.000000,100.00000,64.631735,26.021684,99.9


In [202]:
## CLEANUP DENDRA DATA
# Remove columns where incomplete
dendra_time = dendra_time.loc[:, completeness >= 92]

In [287]:
## JOIN Dendra and WRCC data
weather_time = wrcc_time.join(dendra_time, how="inner")


In [288]:
weather_time['solar_rad_wpm2'].equals(weather_time['total-solar-radiation-avg-wm-2'])

False

In [289]:
weather_time.solar_rad_wpm2.duplicated()


2020-01-01 00:00:00    False
2020-01-01 00:10:00     True
2020-01-01 00:20:00     True
2020-01-01 00:30:00     True
2020-01-01 00:40:00     True
                       ...  
2025-11-10 23:10:00     True
2025-11-10 23:20:00     True
2025-11-10 23:30:00     True
2025-11-10 23:40:00     True
2025-11-10 23:50:00     True
Name: solar_rad_wpm2, Length: 307892, dtype: bool

In [290]:
(weather_time['solar_rad_wpm2'] - weather_time['total-solar-radiation-avg-wm-2']).describe()

count    307889.000000
mean         -6.073588
std         249.887544
min      -11007.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           0.000000
dtype: float64

In [291]:
weather_time['RH_percent'].equals(weather_time['relative-humidity-avg-pct'])


False

In [292]:
(weather_time['RH_percent'] - weather_time['relative-humidity-avg-pct']).describe()


count    307740.000000
mean         -5.949776
std         244.586590
min      -10087.400000
25%          -0.010000
50%           0.000000
75%           0.010000
max           0.050000
dtype: float64

In [293]:
pct_equal = (weather_time['RH_percent'] == weather_time['relative-humidity-avg-pct']).mean() * 100
pct_equal

44.29605186234134

In [294]:
pct_equal = (weather_time['solar_rad_wpm2'] == weather_time['total-solar-radiation-avg-wm-2']).mean() * 100
pct_equal

99.93991399581672

In [295]:
mask_unequal = ~(weather_time['RH_percent'].eq(weather_time['relative-humidity-avg-pct']) | (weather_time['RH_percent'].isna() & weather_time['relative-humidity-avg-pct'].isna()))
unequal_dates = weather_time.index[mask_unequal].sort_values()
groups = (unequal_dates.to_series().diff() != pd.Timedelta("10min")).cumsum()
ranges = unequal_dates.to_series().groupby(groups).agg(start="first", end="last").reset_index(drop=True)
print(ranges)

                    start                 end
0     2020-01-01 08:30:00 2020-01-01 09:20:00
1     2020-01-01 09:40:00 2020-01-01 10:00:00
2     2020-01-01 10:20:00 2020-01-01 12:00:00
3     2020-01-01 12:30:00 2020-01-01 13:00:00
4     2020-01-01 13:30:00 2020-01-01 16:50:00
...                   ...                 ...
19456 2025-11-08 12:40:00 2025-11-08 15:30:00
19457 2025-11-09 09:50:00 2025-11-09 15:20:00
19458 2025-11-10 09:50:00 2025-11-10 12:50:00
19459 2025-11-10 13:30:00 2025-11-10 13:30:00
19460 2025-11-10 13:50:00 2025-11-10 16:30:00

[19461 rows x 2 columns]


In [296]:
mask_close = np.isclose(
    weather_time['solar_rad_wpm2'],
    weather_time['total-solar-radiation-avg-wm-2'],
    rtol=1e-2,
    atol=1e-2,
    equal_nan=True)
pct_close = mask_close.mean() *100
pct_close

99.93991399581672

In [297]:
# Convert precip in inches to centimeters
weather_time['precipitation-geonor-mm'] = weather_time['precip_in'] * 25.4
weather_time.round({'precipitation-geonor-mm':3})

,precip_in,wind_speed_mph,wind_dir_deg,avg_air_temp_F,RH_percent,battery_volts,max_gust_speed_mph,solar_rad_wpm2,air-temp-min-degc,air-temp-max-degc,...,wind-speed-100-ft-avg-ms,wind-speed-25-ft-avg-ms,wind-speed-vector-magnitude-25-ft-ms,wind-speed-vector-magnitude-100-ft-ms,total-solar-radiation-daily-tot-mjm-2,air-temp-25-ft-avg-degc,air-temp-100-ft-avg-degc,relative-humidity-25-ft-avg-pct,relative-humidity-100-ft-avg-pct,precipitation-geonor-mm
2020-01-01 00:00:00,0.0,0.7,274.0,24.7,87.2,13.2,1.8,0.0,-4.306,-3.864,...,1.195,0.325,0.272,1.149,0.0,-2.775,-4.002,76.27,86.3,0.0
2020-01-01 00:10:00,0.0,0.6,229.0,24.9,87.0,13.2,2.0,0.0,-4.102,-3.831,...,1.401,0.288,0.228,1.318,0.0,-2.967,-3.997,73.60,86.1,0.0
2020-01-01 00:20:00,0.0,1.5,232.6,25.2,86.4,13.2,3.0,0.0,-3.933,-3.663,...,2.102,0.660,0.403,2.021,0.0,-2.498,-3.863,69.60,83.2,0.0
2020-01-01 00:30:00,0.0,0.5,211.2,25.6,84.5,13.2,1.8,0.0,-3.663,-3.459,...,1.136,0.237,0.152,1.107,0.0,-2.341,-3.541,72.88,83.7,0.0
2020-01-01 00:40:00,0.0,0.3,265.7,25.7,85.4,13.2,1.7,0.0,-3.528,-3.391,...,0.820,0.156,0.155,0.791,0.0,-2.555,-3.533,72.67,84.2,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-11-10 23:10:00,0.0,0.6,270.8,30.9,98.0,13.2,1.6,0.0,-0.665,-0.604,...,1.485,0.281,0.281,1.471,0.0,-0.587,-0.031,95.80,90.1,0.0
2025-11-10 23:20:00,0.0,0.9,270.5,30.8,98.1,13.2,1.6,0.0,-0.746,-0.633,...,1.634,0.400,0.400,1.604,0.0,-0.626,0.397,95.90,87.1,0.0
2025-11-10 23:30:00,0.0,1.0,269.1,30.6,98.2,13.2,1.8,0.0,-0.846,-0.725,...,1.766,0.451,0.450,1.739,0.0,-0.687,0.214,95.80,88.5,0.0
2025-11-10 23:40:00,0.0,0.9,264.8,30.5,98.2,13.2,1.6,0.0,-0.907,-0.806,...,1.622,0.402,0.402,1.581,0.0,-0.780,0.046,95.90,90.2,0.0


In [298]:
# Remove columns containing substr 'battery'
weather_time = weather_time.loc[:, ~weather_time.columns.str.contains("battery")]
# Remove columns containing substr 'wind_'
weather_time = weather_time.loc[:, ~weather_time.columns.str.contains("wind_")]
# Remove columns containing substr 'RH_'
weather_time = weather_time.loc[:, ~weather_time.columns.str.contains("RH_")]
# Remove columns containing substr 'air_'
weather_time = weather_time.loc[:, ~weather_time.columns.str.contains("air_")]
len(weather_time.columns)

29

In [299]:
weather_time.columns

Index(['precip_in', 'max_gust_speed_mph', 'solar_rad_wpm2',
       'air-temp-min-degc', 'air-temp-max-degc', 'air-temp-avg-degc',
       'barometric-pressure-avg-mb', 'precipitation-geonor-freq-s-1',
       'relative-humidity-max-pct', 'relative-humidity-avg-pct',
       'relative-humidity-min-pct', 'snow-depth-in',
       'total-solar-radiation-avg-wm-2', 'wind-direction-100-ft-std-deg',
       'wind-direction-100-ft-avg-deg', 'wind-direction-25-ft-std-deg',
       'wind-direction-25-ft-avg-deg', 'wind-speed-100-ft-max-ms',
       'wind-speed-25-ft-max-ms', 'wind-speed-100-ft-avg-ms',
       'wind-speed-25-ft-avg-ms', 'wind-speed-vector-magnitude-25-ft-ms',
       'wind-speed-vector-magnitude-100-ft-ms',
       'total-solar-radiation-daily-tot-mjm-2', 'air-temp-25-ft-avg-degc',
       'air-temp-100-ft-avg-degc', 'relative-humidity-25-ft-avg-pct',
       'relative-humidity-100-ft-avg-pct', 'precipitation-geonor-mm'],
      dtype='object')

In [303]:
weather_time = weather_time.loc[:,['air-temp-avg-degc','air-temp-min-degc','air-temp-max-degc','air-temp-25-ft-avg-degc','air-temp-100-ft-avg-degc',
                                   'barometric-pressure-avg-mb','precipitation-geonor-mm','precipitation-geonor-freq-s-1','snow-depth-in',
                                   'relative-humidity-avg-pct','relative-humidity-min-pct','relative-humidity-max-pct','relative-humidity-25-ft-avg-pct','relative-humidity-100-ft-avg-pct',
                                   'total-solar-radiation-avg-wm-2','total-solar-radiation-daily-tot-mjm-2',
                                   'wind-speed-25-ft-avg-ms','wind-speed-25-ft-max-ms','wind-direction-25-ft-avg-deg','wind-direction-25-ft-std-deg','wind-speed-vector-magnitude-25-ft-ms',
                                   'wind-speed-100-ft-avg-ms','wind-speed-100-ft-max-ms','wind-direction-100-ft-avg-deg','wind-direction-100-ft-std-deg','wind-speed-vector-magnitude-100-ft-ms',]]
weather_time.columns

Index(['air-temp-avg-degc', 'air-temp-min-degc', 'air-temp-max-degc',
       'air-temp-25-ft-avg-degc', 'air-temp-100-ft-avg-degc',
       'barometric-pressure-avg-mb', 'precipitation-geonor-mm',
       'precipitation-geonor-freq-s-1', 'snow-depth-in',
       'relative-humidity-avg-pct', 'relative-humidity-min-pct',
       'relative-humidity-max-pct', 'relative-humidity-25-ft-avg-pct',
       'relative-humidity-100-ft-avg-pct', 'total-solar-radiation-avg-wm-2',
       'total-solar-radiation-daily-tot-mjm-2', 'wind-speed-25-ft-avg-ms',
       'wind-speed-25-ft-max-ms', 'wind-direction-25-ft-avg-deg',
       'wind-direction-25-ft-std-deg', 'wind-speed-vector-magnitude-25-ft-ms',
       'wind-speed-100-ft-avg-ms', 'wind-speed-100-ft-max-ms',
       'wind-direction-100-ft-avg-deg', 'wind-direction-100-ft-std-deg',
       'wind-speed-vector-magnitude-100-ft-ms'],
      dtype='object')

In [304]:
len(weather_time.columns)

26

In [305]:
weather_time.to_csv('data/station_instrumentation/climate/2010_2025_10min_SagehenWeather.csv')